In [ ]:
!pip install rawpy imageio exifread

from google.colab import files
uploaded = files.upload()

In [ ]:
# Finding defective pixel by pixel to pixel itteration (slow)

import rawpy
import numpy as np
from PIL import Image
import cv2

################################################################################
def load_image(file_name= 'a0151-JI2E3955.dng', sample_size = 100):
    """
    """

    with rawpy.imread(file_name) as raw:
        pattern = raw.raw_pattern
        bayer_img = raw.raw_image_visible
        color_map = {0: 'R', 1: 'G', 2: 'B', 3: 'G'}
        named_pattern = np.vectorize(color_map.get)(pattern)
        print(named_pattern)

        # select sample section of the image
        H, W = bayer_img.shape
        CY, CX = H//2, W//2
        sample = bayer_img[CY:CY+sample_size, CX:CX+sample_size]

        # Convert embedded thumbnail to NumPy array
        thumbnail = raw.extract_thumb()
        print(thumbnail.format) # we this later
        if thumbnail.format == rawpy.ThumbFormat.JPEG:
            img = Image.open(io.BytesIO(thumbnail.data))
            thumbnail_np = np.array(img.convert("L"))  # convert to grayscale
        elif thumbnail.format == rawpy.ThumbFormat.BITMAP:
            thumbnail_np = np.array(thumbnail.data)
        else:
            raise ValueError("Unknown thumbnail format")

    return bayer_img, sample, thumbnail_np

################################################################################
def get_neighbors_RGGB(img, y, x, offset = 4) :

    """
    Finds the neighboring pixels for a given pixel in a Bayer image.
    """
    H, W = img.shape

    if offset == 8:
        offset_list = [(-4,0), (-2,0), (2,0), (4,0), (-2,-2), (-2,2), (2,2), (2,-2)]
    else:
        offset_list = [(-2,0), (0,-2), (2,0), (0,2)]

    neighbors = []
    for dx, dy in offset_list:
        ny, nx = y + dy, x + dx
        if 0 <= ny < H and 0 <= nx < W:
            neighbors.append(bayer_img[ny, nx])

    return neighbors

################################################################################
def DPC(bayer_img, threshold= 50, offset = 4):
    """
    Detects and corrects defective pixels in a raw Bayer image.

    Parameters:
        bayer_img (np.ndarray): 2D Bayer image (grayscale).
        threshold (int): intensity difference threshold for detecting outliers.

    Returns:
        corrected_img (np.ndarray): image with defective pixels corrected.
        num_defective (int): number of defective pixels found and corrected.
    """
    H, W = bayer_img.shape
    corrected_img = bayer_img.copy()
    num_defective = 0
    defective_mask = np.zeros_like(bayer_img, dtype=bool)

    margin = 4 if offset == 8 else 2
    for y in range(margin, H - margin):
        for x in range(margin, W - margin):
            neighbors = get_neighbors_RGGB(bayer_img, y, x, offset)
            medial_val = np.median(neighbors)
            if abs(bayer_img[y,x] - medial_val) > threshold:
                corrected_img[y, x] = int(np.mean(neighbors))
                num_defective += 1
                defective_mask[y,x] = True

    return corrected_img.astype(np.uint16), num_defective, defective_mask

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np # Import numpy
from PIL import Image # Import PIL Image
import io # Import io

bayer_img, sample, thumbnail = load_image(file_name= 'a0151-JI2E3955.dng', sample_size= 200)

fixed_img, count, mask = DPC(sample, threshold= 80, offset = 4)
print('Defective pixels found= ', count)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(sample, cmap='gray')
axes[0].set_title('Original Image Sample')

axes[1].imshow(mask, cmap='gray')
axes[1].set_title('DFC Mask')

axes[2].imshow(thumbnail, cmap='gray')
axes[2].set_title('Thumbnail')

for ax in axes:
    ax.axis('off')
plt.show()

In [ ]:
import numpy as np
import cv2

def fast_DPC(bayer_img, threshold=50, ksize=3):
    """
    Fast defective pixel correction using median filtering.
    Applies a median filter and replaces outliers based on a threshold.

    Parameters:
        bayer_img (np.ndarray): 2D Bayer image.
        threshold (int): Pixel difference threshold.
        ksize (int): Median filter kernel size (must be odd).

    Returns:
        corrected_img, num_defective, defect_mask
    """
    assert ksize % 2 == 1, "ksize must be odd"

    # Step 1: Compute median image
    median_img = cv2.medianBlur(bayer_img, ksize)

    # Step 2: Identify defective pixels
    diff = np.abs(bayer_img.astype(np.int16) - median_img.astype(np.int16))
    defect_mask = diff > threshold

    # Step 3: Replace defects with median
    corrected_img = bayer_img.copy()
    corrected_img[defect_mask] = median_img[defect_mask]

    num_defective = np.sum(defect_mask)

    return corrected_img.astype(np.uint16), num_defective, defect_mask

In [ ]:
bayer_img, sample, thumbnail = load_image(file_name= 'a0151-JI2E3955.dng')

corrected, count, mask = fast_DPC(sample, threshold=80, ksize=3)
print("Defective pixels corrected:", count)